# Fitness function

A function that estimates the success of a single speciment - it will allow for survival of the fittest, meaning that the successful speciment will have a higher chance to be picked and reproduce into another generation.

In this case we create a score between 0 and 100 as to how close an estimate string is to the real thing.

In [1]:
def fitness(input_word, test_word):
    if len(input_word) != len(test_word):
        print('Input word and test word are not of equal length!')
        return
    
    score = 0
    for index, char in enumerate(test_word):
        if char == input_word[index]:
            score += 1
            
    return score * 100 / len(input_word)

# Create individuals

Taking its influence from human evolution and DNA, Genetic Programming roughly operates in the similar way.
DNA is composed of **genes** and each gene has its variant form called **allele**. In out case, since we are trying to figure out a string based on evolution of randomly selected characters, each letter is a gene, and the value of each particular word is allele - the whole string would be the DNA.
Thus, the word *python* is a DNA, each letter is a gene, and the first gene *p* is an allele of that gene.

In our case, we are assuming we already know the length of the string we want to recreate, and the population is every word that can be composed using alphabetic character in the same length. Genetic algorithm will explore all possible combinations.

The main idea when creating a first population is that we shouldn't point the algorithm in a corrent direction right away - instead the first population should cover as wide range of possibilities as possible. The perfect population should cover every possible allele, but obviously that's not doable due to the lack of memory and computing power. Instead, we will generate random letter in place of each charater of a specified length.

In [2]:
import random
import string

def generate_word(length):
    return ''.join(random.choice(string.ascii_letters) for _ in range(length))

def generate_initial_population(population_size, input_word_length):
    return [generate_word(input_word_length) for _ in range(population_size)]

# Breeding

Breeding occurs in the way that 2 parents are chosen based on their fitness score and parts of each parent is taken at random (or using some breeding techniques) to create and offspring.

## Parents selection

For parents, usually the best fitted individuals will be selected, but it's important to not discard other completely as part of their DNA might be crucial for the 'survival'. In other ways choosing only the good solutions in the begining will cause the algorithm to converge quickly toward local minimum due to the restricted amount of information available, instead of finding the best possible solution.

One way to do that is to create a perfect population and choose N samples from there, but also to allow for a specific number of lucky few individuals without any discriminations to their fitness levels.

In [3]:
def compute_perfect_population(population, input_word):
    most_fitted = []
    for individual in population:
        most_fitted.append((individual, fitness(input_word, individual)))
    
    return sorted(most_fitted, key=lambda x: x[1], reverse=True)

def select_parents(most_fitted, num_best, num_lucky):
    parents = []
    for i in range(num_best):
        parents.append(most_fitted[i][0])
    for i in range(num_lucky):
        parents.append(random.choice(most_fitted)[0])
    
    random.shuffle(parents)
    return parents

## Mutation

To make things more interested and to prevent the algorithm from stopping in local minimum (or to increase the chances of finding the global optimum), we will introduce a small probability for each offspring that one of its alleles (letters) will mutate - meaning it will be replaced by a randomly chosen letter.

In [4]:
def mutate_individual(individual):
    index = int(random.random() * len(individual))
    if (index == 0):
        return random.choice(string.ascii_letters) + individual[1:]
    else:
        return individual[:index] + random.choice(string.ascii_letters) + individual[index+1:]

## Offspring generation

As mentioned above, the idea of breeding is to mix the DNA of parents in order to create an offspring that gets its parents' features. The DNA of each parent is defined by their alleles (letters) - in order to mix DNA we just need to mix the alleles of each parent.

There are many ways to do this. In this case we're going for the simpliest solution - for each letter of the child, take at random one of the letters from either parent. Of course we'll have to create a similar regulation to 'one'-child policy, otherwise the population will expload in exponentially. To do so, let's restrict he number of children to the number of individuals in the first generation. When creating children, we will also provide a small probability for mutation for each child.

In [5]:
def create_child(individual_1, individual_2):
    assert len(individual_1) == len(individual_2)
    return ''.join(individual_1[i] if random.random() < .5 else individual_2[i] for i in range(len(individual_1)))

def create_children(parents, num_children, mutation_chance=.05):
    next_population = []
    for _ in range(num_children):
        parent_1 = random.choice(parents)
        parent_2 = random.choice(parents)
        child = create_child(parent_1, parent_2)
        
        if random.random() < mutation_chance:
            child = mutate_individual(child)
            
        next_population.append(child)
        
    return next_population

# Evolution

Time to put together all the functions we've created and run the evolution! Set some hyperparameters and watch the *biology* do its job in recreating the input string.

In [6]:
import pandas as pd

POPULATION_SIZE = 100
MUTATION_CHANCE = .05
LUCKY_FEW = int(.25 * POPULATION_SIZE)
NUM_BEST = POPULATION_SIZE - LUCKY_FEW -1
NUM_GENERATIONS = 1000
INPUT_STRING = 'armageddon'

In [12]:
history = []
initial_population = generate_initial_population(POPULATION_SIZE, len(INPUT_STRING))

In [13]:
def run_evolution(curr_gen, population):
    most_fitted = compute_perfect_population(population, INPUT_STRING)
    print('Best effort in generation #{} is \'{}\' with a score of: {}'.format(curr_gen, most_fitted[0][0], most_fitted[0][1]))
    history.append((curr_gen, most_fitted[0][1], most_fitted[0][0]))

    parents = select_parents(most_fitted, NUM_BEST, LUCKY_FEW)
    children = create_children(parents, POPULATION_SIZE)
    
    if most_fitted[0][1] >= 100:
        print('Achieved 100% replication in {} generations!'.format(curr_gen))
        return
    
    curr_gen += 1
    if curr_gen < NUM_GENERATIONS:
        run_evolution(curr_gen, children)
    else:
        return most_fitted

In [14]:
most_fitted = run_evolution(0, initial_population)

Best effort in generation #0 is 'aLmHpYnXyn' with a score of: 30.0
Best effort in generation #1 is 'JLLbgaFXon' with a score of: 30.0
Best effort in generation #2 is 'JLcogLbuon' with a score of: 30.0
Best effort in generation #3 is 'YLcoggZdon' with a score of: 40.0
Best effort in generation #4 is 'aNcowgZdon' with a score of: 40.0
Best effort in generation #5 is 'aNmwaepXwk' with a score of: 30.0
Best effort in generation #6 is 'ulBagsqdon' with a score of: 50.0
Best effort in generation #7 is 'ayiZaeFdLn' with a score of: 40.0
Best effort in generation #8 is 'MymZgeFdLn' with a score of: 50.0
Best effort in generation #9 is 'MLgogeFdon' with a score of: 50.0
Best effort in generation #10 is 'aNmbgkilon' with a score of: 50.0
Best effort in generation #11 is 'aXqbgeFdon' with a score of: 60.0
Best effort in generation #12 is 'uNmagSadon' with a score of: 60.0
Best effort in generation #13 is 'aygKgeDdon' with a score of: 60.0
Best effort in generation #14 is 'ayTZgeDdon' with a score

# Some extra visualization

Use bokeh to visualize the training with most fitted individual from each generation.

In [15]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, LabelSet, Label

output_notebook()

Loading BokehJS ...

In [16]:
generation, score, individual = zip(*history)

source = ColumnDataSource({
    'generation': generation, 
    'score': score, 
    'individual': individual}
)
labels_source = ColumnDataSource({
    'generation': generation[::50], 
    'score': score[::50], 
    'individual': individual[::50]}
)

TOOLTIPS = [
    ('Best attempt:', '@individual'),
    ('Generation:', '@generation'),
    ('Score:', '@score')
]

hover = HoverTool(
    tooltips=TOOLTIPS,
    mode='vline'
)

labels = LabelSet(x='generation', y='score', text='individual', level='glyph', angle=.4,
                  x_offset=15, y_offset=5, source=labels_source, render_mode='canvas')

p = figure(plot_width=800, 
           plot_height=500, 
           tools=[hover],
           title='Best fitted individual in each generation.')

p.line(x='generation', y='score', source=source, line_width=2)
p.xaxis.axis_label = 'Generation'
p.yaxis.axis_label = 'Score'
p.add_layout(labels)

show(p)